# Filtering Microcap Alts

In [1]:
#%% Importing modules and data
import smtplib
import pandas as pd
import numpy as np
import datetime as dt
import pandas.stats.moments as st
from pandas import ExcelWriter
import matplotlib.pyplot as plt
import os
import quandl as qd
import seaborn as sns
import matplotlib.dates as dates
import matplotlib.ticker as ticker
from pandas_datareader import data
from lxml import html
import requests
import webbrowser
from bs4 import BeautifulSoup as bs
import json
import csv
import os
import glob
from time import sleep


# Functions for pulling from Coinmarketcap
- coindfpull: pulls coin information from cmc including total coin supply
- tokendfpull: pulls token information from cmc including total token supply
- coincircdf: pulls coin circulating supply from cmc
- tokencircdf: pulls token circulating supply from cmc

In [2]:
def coindfpull(pagenum):
    site = 'https://coinmarketcap.com/coins/views/market-cap-by-total-supply/' + str(pagenum)
    res = requests.get(site)
    if str(res) == "<Response [404]>":
        return "404 Response"
    
    soup = bs(res.text, 'lxml')

    table = soup.find_all('table')[0] 

    ranks = []
    currnames = []
    mcapsusd = []
    mcapsbtc = []
    pricesusd = []
    pricesbtc = []
    volumes24husd = []
    volumes24hbtc = []
    totalsupplies = []
    #changes24husd = []
    #changes24hbtc = []

    for row in table.find_all('tr'):
        columns = str(row).split('\n')
        if '#' in columns[1]:
            continue
        else:
            ranks = ranks + [columns[2].replace(' ','')]
            currnames = currnames + [columns[6].split('<span class="currency-symbol"><a href="/currencies/')[1].split('/')[0]]
            mcapsusd = mcapsusd + [columns[9].split('data-btc="')[1].split(' " data-usd="')[1].replace('">','')]
            mcapsbtc = mcapsbtc + [columns[9].split('data-btc="')[1].split(' " data-usd="')[0]]
            pricesusd = pricesusd + [columns[15].split('" data-btc="')[1].split('" data-usd="')[1].split('" ')[0]]
            pricesbtc = pricesbtc + [columns[15].split('" data-btc="')[1].split('" data-usd="')[0]]
            volumes24husd = volumes24husd + [columns[18].split('" data-btc="')[1].split('" data-usd="')[1].split('" ')[0]]
            volumes24hbtc = volumes24hbtc + [columns[18].split('" data-btc="')[1].split('" data-usd="')[0]]
            totalsupplies = totalsupplies + [columns[21].split('data-supply="')[1].split('" ')[0].replace('">','')]
            #changes24husd = changes24husd + [columns[-3].split('" data-btc="')[1].split('" data-usd="')[1].split('">')[0]]
            #changes24hbtc = changes24hbtc + [columns[-3].split('" data-btc="')[1].split('" data-usd="')[0]]
    
    crypto_dict = {"Rank": ranks,
                   "Names": currnames,
                   "MktCapUSD": mcapsusd,
                   "MktCapBTC": mcapsbtc,
                   "PricesUSD": pricesusd,
                   "PricesBTC": pricesbtc,
                   "Vol24USD": volumes24husd,
                   "Vol24BTC": volumes24hbtc,
                   "TotalSupply": totalsupplies}
                   #"Change24USD": changes24husd,
                   #"Change24BTC": changes24hbtc
                  #}
    
    outdf = pd.DataFrame(crypto_dict, index = currnames)
    
    return outdf

def tokendfpull(pagenum):
    site = 'https://coinmarketcap.com/tokens/views/market-cap-by-total-supply/' + str(pagenum)
    res = requests.get(site)
    if str(res) == "<Response [404]>":
        return "404 Response"
    
    soup = bs(res.text, 'lxml')

    table = soup.find_all('table')[0] 

    ranks = []
    currnames = []
    platforms = []
    mcapsusd = []
    mcapsbtc = []
    pricesusd = []
    pricesbtc = []
    volumes24husd = []
    volumes24hbtc = []
    totalsupplies = []
    #changes24husd = []
    #changes24hbtc = []

    for row in table.find_all('tr'):
        columns = str(row).split('\n')
        if '#' in columns[1]:
            continue
        else:
            ranks = ranks + [columns[2].replace(' ','')]
            platforms = platforms + [columns[0].split('data-platformsymbol="')[1].split('" ')[0]]
            currnames = currnames + [columns[6].split('<span class="currency-symbol"><a href="/currencies/')[1].split('/')[0]]
            mcapsusd = mcapsusd + [columns[10].split('data-btc="')[1].split(' " ')[1].split('data-usd="')[1].replace('">','')]
            mcapsbtc = mcapsbtc + [columns[10].split('data-btc="')[1].split(' " ')[0]]
            pricesusd = pricesusd + [columns[16].split('data-btc="')[1].split('" ')[2].replace('data-usd="','')]
            pricesbtc = pricesbtc + [columns[16].split('data-btc="')[1].split('" ')[0]]
            volumes24husd = volumes24husd + [columns[19].split('data-btc="')[1].split('" ')[2].replace('data-usd="','')]
            volumes24hbtc = volumes24hbtc + [columns[19].split('data-btc="')[1].split('" ')[0]]
            totalsupplies = totalsupplies + [columns[22].split('data-supply="')[1].split('" ')[0].replace('">','')]
            #changes24husd = changes24husd + [columns[-3].split('" data-btc="')[1].split('" data-usd="')[1].split('">')[0]]
            #changes24hbtc = changes24hbtc + [columns[-3].split('" data-btc="')[1].split('" data-usd="')[0].split('" ')[0]]
    
    crypto_dict = {"Rank": ranks,
                   "Names": currnames,
                   "Platform": platforms,
                   "MktCapUSD": mcapsusd,
                   "MktCapBTC": mcapsbtc,
                   "PricesUSD": pricesusd,
                   "PricesBTC": pricesbtc,
                   "Vol24USD": volumes24husd,
                   "Vol24BTC": volumes24hbtc,
                   "TotalSupply": totalsupplies}
                   #"Change24USD": changes24husd,
                   #"Change24BTC": changes24hbtc
                  #}
    
    outdf = pd.DataFrame(crypto_dict, index = currnames)
    
    return outdf

def coincircdf(pagenum):
    site = 'https://coinmarketcap.com/coins/' + str(pagenum)
    res = requests.get(site)
    if str(res) == "<Response [404]>":
        return "404 Response"
    
    soup = bs(res.text, 'lxml')

    table = soup.find_all('table')[0] 
    
    currnames = []
    circulatingsupplies = []

    for row in table.find_all('tr'):
        columns = str(row).split('\n')
        if '#' in columns[1]:
            continue
        else:
            currnames = currnames + [columns[6].split('<span class="currency-symbol"><a href="/currencies/')[1].split('/')[0]]
            circulatingsupplies = circulatingsupplies + [columns[21].split('data-supply="')[1].split('" ')[0].replace('">','')]
    
    crypto_dict = {"Names": currnames,
                   "CirculatingSupply": circulatingsupplies}
    
    outdf = pd.DataFrame(crypto_dict, index = currnames)
    del outdf['Names']
    
    return outdf

def tokencirdf(pagenum):
    site = 'https://coinmarketcap.com/tokens/' + str(pagenum)
    res = requests.get(site)
    if str(res) == "<Response [404]>":
        return "404 Response"
    
    soup = bs(res.text, 'lxml')

    table = soup.find_all('table')[0] 
    
    currnames = []
    circulatingsupplies = []

    for row in table.find_all('tr'):
        columns = str(row).split('\n')
        if '#' in columns[1]:
            continue
        else:
            currnames = currnames + [columns[6].split('<span class="currency-symbol"><a href="/currencies/')[1].split('/')[0]]
            circulatingsupplies = circulatingsupplies + [columns[22].split('data-supply="')[1].split('" ')[0].replace('">','')]
    
    crypto_dict = {"Names": currnames,
                   "CirculatingSupply": circulatingsupplies}
    
    outdf = pd.DataFrame(crypto_dict, index = currnames)
    del outdf['Names']
    
    return outdf

# Alternative Function for Pulling all Coin/Token Caps

In [3]:
def pullfulldf():
    site = 'https://coinmarketcap.com/all/views/all/'
    res = requests.get(site)
    soup = bs(requests.get(site).text, "lxml")

    table_start = '<table class="table js-summary-table dataTable no-footer" id="currencies-all" style="font-size: 14px; width: 100%;" role="grid">'
    table_end = '</table>'

    table = soup.find_all('table')[0]

    symbol = []
    cname = []
    mcapsusd = []
    pricesusd = []
    pricesbtc = []
    volumes24husd = []
    totalsupplies = []
    availablesupplies = []
    maxsupplies = []
    change1h = []
    change24h = []
    change7d = []
    i = 1
    for row in table.find_all('tr'):
        columns = str(row).split('\n')
        if not('class' in columns[0]):
            continue

        cmcname = columns[0].split('"id-')[1].replace('">','')

        apisite = 'https://api.coinmarketcap.com/v1/ticker/' + cmcname + '/'
        response = requests.get(apisite)
        json_data = json.loads(response.text)

        cname = cname + [cmcname]
        symbol = symbol + [json_data[0]['symbol']]
        mcapsusd = mcapsusd + [json_data[0]['market_cap_usd']]
        pricesusd = pricesusd + [json_data[0]['price_usd']]
        pricesbtc = pricesbtc + [json_data[0]['price_btc']]
        volumes24husd = volumes24husd + [json_data[0]['24h_volume_usd']]
        totalsupplies = totalsupplies + [json_data[0]['total_supply']]
        availablesupplies = availablesupplies + [json_data[0]['available_supply']]
        maxsupplies = maxsupplies + [json_data[0]['max_supply']]
        change1h = change1h + [json_data[0]['percent_change_1h']]
        change24h = change24h + [json_data[0]['percent_change_24h']]
        change7d = change7d + [json_data[0]['percent_change_7d']]
        
        print('Number: ' + str(i) + ' Coin: ' + json_data[0]['symbol'])
        sleep(1)
        i += 1

    fulldict = {'CMCname': cname,
                'Symbol': symbol,
                'Mcap': mcapsusd,
                'USDPrice': pricesusd,
                'BTCPrice': pricesbtc,           
                'Vol24h': volumes24husd,           
                'TotalSupply': totalsupplies,           
                'AvailableSupply': availablesupplies,    
                'MaxSupply': maxsupplies,
                'Change1h': change1h,
                'Change24h': change24h,
                'Change7d': change7d}
    fulldf = pd.DataFrame(fulldict,index = cname)
    
    return fulldf


# Creating dataframe for initial filtering
- Keep only marketcaps with less than 100M
- Remove tokens with 24 hour volumes less than 2% of marketcaps
- Remove tokens with circulating supply/total supply with less than 20%

In [4]:
# Creating DF for Coins

def microfilter1(mcapthreshold, volthreshold, supplythreshold):
    pagenmbr = 1
    coinlist = []
    while type(coindfpull(pagenmbr)) != str:
        coinlist = coinlist + [coindfpull(pagenmbr)]
        pagenmbr += 1

    pagenmbr = 1
    coincirclist = []
    while type(coincircdf(pagenmbr)) != str:
        coincirclist = coincirclist + [coincircdf(pagenmbr)]
        pagenmbr += 1

    coinscirc = pd.concat(coincirclist, axis = 0)
    coins = pd.concat(coinlist, axis = 0)
    coins = coins[coins['MktCapUSD'] != '?']
    coins = pd.concat([coins.drop_duplicates(),coinscirc.drop_duplicates()], axis = 1, join = 'inner')

    # Creating DF for Tokens

    pagenmbr = 1
    tokenlist = []
    while type(tokendfpull(pagenmbr)) != str:
        tokenlist = tokenlist + [tokendfpull(pagenmbr)]
        pagenmbr += 1

    pagenmbr = 1
    tokencirclist = []
    while type(tokencirdf(pagenmbr)) != str:
        tokencirclist = tokencirclist + [tokencirdf(pagenmbr)]
        pagenmbr += 1

    tokenscirc = pd.concat(tokencirclist, axis = 0)
    tokens = pd.concat(tokenlist, axis = 0)
    tokens = tokens[tokens['MktCapUSD'] != '?']
    tokens = pd.concat([tokens.drop_duplicates(),tokenscirc.drop_duplicates()], axis = 1, join = 'inner')

    totaldf = pd.concat([coins,tokens], axis = 0)
    totaldf = totaldf[totaldf['CirculatingSupply'] != 'None']
    totaldf = totaldf[totaldf['Vol24USD'] != 'None']

    totaldf['MktCapUSD'] = pd.to_numeric(totaldf['MktCapUSD'])
    totaldf['CirculatingSupply'] = pd.to_numeric(totaldf['CirculatingSupply'])
    totaldf['TotalSupply'] = pd.to_numeric(totaldf['TotalSupply'])
    totaldf['Vol24USD'] = pd.to_numeric(totaldf['Vol24USD'])
    totaldf['VolRatio'] = totaldf['Vol24USD']/totaldf['MktCapUSD']
    totaldf['SupplyRatio'] = totaldf['CirculatingSupply']/totaldf['TotalSupply']
    
    totaldf = totaldf[totaldf['MktCapUSD'] <= mcapthreshold]
    totaldf = totaldf[totaldf['VolRatio'] >= volthreshold]
    totaldf = totaldf[totaldf['SupplyRatio'] >= supplythreshold]
    
    
    return totaldf

In [11]:
def checkannouncement(cmcname):

    site = 'https://coinmarketcap.com/currencies/' + cmcname + '/'

    res = requests.get(site)
    soup = str(bs(res.text, 'lxml'))

    if "Announcement" in soup:
        announcementsite = soup.split('<li><span class="glyphicon glyphicon-bullhorn text-gray" title="Announcement">')[1].split('<a href="')[1].split('" target')[0]
    else:
        announcementsite = "NA"

    if 'bitcointalk.org' in announcementsite:
        return 1, announcementsite
    else:
        return 0, 'NA'
    
def bctcheck(cmcname):

    annsite = checkannouncement(cmcname)[1]
    parsesoup = annsite.split('=')[0] + '=' + annsite.split('=')[1].split('.')[0]
    res = requests.get(parsesoup)
    soup = str(bs(res.text, 'lxml'))
    discussion = soup.split('<td class="middletext"')[1].split('<span class="prevnext">')[0]
    discussion = discussion.split('<div class="nav"')[0]
    if 'All' in discussion.split(parsesoup)[-1]:
        lastdiscuss = discussion.split(parsesoup)[-2]
    else:
        lastdiscuss = discussion.split(parsesoup)[-1]
        
    if '<td align="right" style="padding-right:' in discussion:
        lastpgIndex = lastdiscuss.split('">')[0]
    else:
        lastpgIndex = lastdiscuss.split('">')[0]

    lastpg = parsesoup + lastpgIndex
    res = requests.get(lastpg)
    soup = str(bs(res.text, 'lxml'))
    totpages = soup.split('<td class="middletext"')[1].split('[<b>')[1].split('</b>]')[0]

    forumparse = lastpg.split('=')[1]
    res = requests.get(lastpg)
    soup = str(bs(res.text, 'lxml'))
    lastmsg = soup.split(forumparse)[1].split('<tbody>')[-1].split('<div class="smalltext">')[-1].split('</div>')[0]
    
    return totpages, lastmsg

In [ ]:
'''
# Using old method to build dataframe, no change fields available

cmcdf = microfilter1(100000000, 0.02, 0.1)

names = list(cmcdf.index)
announcements = []
bctpages = []
lastentries = []

for cmcname in names:
    bctentry, annsite = checkannouncement(cmcname)
    announcements = announcements + [bctentry]
    if bctentry == 1:
        totpgs, last = bctcheck(cmcname)
    else:
        totpgs, last = (0,0)
        
    bctpages = bctpages + [totpgs]
    lastentries = lastentries + [last]
    print('Name: ' + str(len(bctpages)) + ' Alt: ' + cmcname)
    sleep(1)
    
bctdf = pd.DataFrame({'bctpages': bctpages,
                     'lastentry': lastentries},
                    index = names)
filterdf = pd.concat([cmcdf,bctdf],axis = 1)
filterdf.to_csv('cmc2.csv')

'''

In [11]:
cmcfull = pullfulldf()

names = list(cmcfull.index)
announcements = []
bctpages = []
lastentries = []

for cmcname in names:
    bctentry, annsite = checkannouncement(cmcname)
    announcements = announcements + [bctentry]
    if bctentry == 1:
        try:
            totpgs, last = bctcheck(cmcname)
        except:
            totpgs, last = (0,0)
        
    else:
        totpgs, last = (0,0)
        
    bctpages = bctpages + [totpgs]
    lastentries = lastentries + [last]
    print('Name: ' + str(len(bctpages)) + ' Alt: ' + cmcname)
    sleep(1)
    
bctdf = pd.DataFrame({'bctpages': bctpages,
                     'lastentry': lastentries},
                    index = names)
filterdf = pd.concat([cmcfull,bctdf],axis = 1)

binance = pd.read_csv('binancelist.csv')
binance.index = binance['Name']
del binance['Ticker'], binance['Name']

bittrex = pd.read_excel('bittrexlist.xlsx')
bittrex.index = bittrex['Name']
del bittrex['Ticker'], bittrex['Name']

kucoin = pd.read_csv('kucoinlist.csv')
kucoin.index = kucoin['Name']
del kucoin['Ticker'], kucoin['Name']

cryptopia = pd.read_csv('cryptopialist.csv')
cryptopia.index = cryptopia['Name']
del cryptopia['Ticker'], cryptopia['Name']

fulldf = pd.concat([filterdf,cryptopia,kucoin,binance,bittrex],axis = 1)
fulldf.head()

fulldf.to_csv('cmc.csv')

Number: 1 Coin: BTC
Number: 2 Coin: ETH
Number: 3 Coin: XRP
Number: 4 Coin: BCH
Number: 5 Coin: ADA
Number: 6 Coin: XLM
Number: 7 Coin: NEO
Number: 8 Coin: LTC
Number: 9 Coin: EOS
Number: 10 Coin: XEM
Number: 11 Coin: MIOTA
Number: 12 Coin: DASH
Number: 13 Coin: XMR
Number: 14 Coin: TRX
Number: 15 Coin: QTUM
Number: 16 Coin: LSK
Number: 17 Coin: ETC
Number: 18 Coin: VEN
Number: 19 Coin: USDT
Number: 20 Coin: BTG
Number: 21 Coin: ICX
Number: 22 Coin: XRB
Number: 23 Coin: PPT
Number: 24 Coin: OMG
Number: 25 Coin: ZEC
Number: 26 Coin: STEEM
Number: 27 Coin: BTS
Number: 28 Coin: STRAT
Number: 29 Coin: BCN
Number: 30 Coin: BNB
Number: 31 Coin: SC
Number: 32 Coin: XVG
Number: 33 Coin: ZRX
Number: 34 Coin: MKR
Number: 35 Coin: VERI
Number: 36 Coin: SNT
Number: 37 Coin: REP
Number: 38 Coin: WAVES
Number: 39 Coin: WTC
Number: 40 Coin: AE
Number: 41 Coin: RHOC
Number: 42 Coin: KCS
Number: 43 Coin: ARDR
Number: 44 Coin: DOGE
Number: 45 Coin: DCR
Number: 46 Coin: DGD
Number: 47 Coin: KMD
Number: 4

Number: 373 Coin: UNIT
Number: 374 Coin: RNT
Number: 375 Coin: DIVX
Number: 376 Coin: AIT
Number: 377 Coin: XPM
Number: 378 Coin: LUX
Number: 379 Coin: ICOS
Number: 380 Coin: PLBT
Number: 381 Coin: BQ
Number: 382 Coin: FLO
Number: 383 Coin: SXUT
Number: 384 Coin: XAUR
Number: 385 Coin: IFT
Number: 386 Coin: AURA
Number: 387 Coin: HDG
Number: 388 Coin: SPF
Number: 389 Coin: TIME
Number: 390 Coin: AVT
Number: 391 Coin: FLDC
Number: 392 Coin: UQC
Number: 393 Coin: LMC
Number: 394 Coin: B2B
Number: 395 Coin: LEV
Number: 396 Coin: QAU
Number: 397 Coin: DBET
Number: 398 Coin: HEAT
Number: 399 Coin: AIX
Number: 400 Coin: RIC
Number: 401 Coin: DTB
Number: 402 Coin: OBITS
Number: 403 Coin: GBX
Number: 404 Coin: GET
Number: 405 Coin: ATB
Number: 406 Coin: XUC
Number: 407 Coin: PND
Number: 408 Coin: SYNX
Number: 409 Coin: INCNT
Number: 410 Coin: LOC
Number: 411 Coin: COLX
Number: 412 Coin: DOVU
Number: 413 Coin: HKN
Number: 414 Coin: NIO
Number: 415 Coin: PIRL
Number: 416 Coin: RC
Number: 417 Coi

Number: 739 Coin: NYAN
Number: 740 Coin: VISIO
Number: 741 Coin: CFT
Number: 742 Coin: UNIC
Number: 743 Coin: CFD
Number: 744 Coin: CPC
Number: 745 Coin: MCRN
Number: 746 Coin: MNE
Number: 747 Coin: KURT
Number: 748 Coin: UTC
Number: 749 Coin: MDC
Number: 750 Coin: TKR
Number: 751 Coin: NETKO
Number: 752 Coin: RLT
Number: 753 Coin: HTC
Number: 754 Coin: WTT
Number: 755 Coin: TRI
Number: 756 Coin: MOTO
Number: 757 Coin: EL
Number: 758 Coin: GUN
Number: 759 Coin: TROLL
Number: 760 Coin: IETH
Number: 761 Coin: ATOM
Number: 762 Coin: XJO
Number: 763 Coin: VIDZ
Number: 764 Coin: WGO
Number: 765 Coin: WHL
Number: 766 Coin: GLT
Number: 767 Coin: OTX
Number: 768 Coin: DAXX
Number: 769 Coin: OPAL
Number: 770 Coin: XBL
Number: 771 Coin: BIGUP
Number: 772 Coin: TRUMP
Number: 773 Coin: FUNC
Number: 774 Coin: HERO
Number: 775 Coin: SLG
Number: 776 Coin: XPTX
Number: 777 Coin: TIT
Number: 778 Coin: BRIT
Number: 779 Coin: PAK
Number: 780 Coin: JNS
Number: 781 Coin: NUKO
Number: 782 Coin: CASH
Number:

Number: 1097 Coin: PIE
Number: 1098 Coin: CWXT
Number: 1099 Coin: MILO
Number: 1100 Coin: CESC
Number: 1101 Coin: VIP
Number: 1102 Coin: WARP
Number: 1103 Coin: LTCR
Number: 1104 Coin: PULSE
Number: 1105 Coin: RIDE
Number: 1106 Coin: ATX
Number: 1107 Coin: PONZI
Number: 1108 Coin: TAGR
Number: 1109 Coin: PLNC
Number: 1110 Coin: TOR
Number: 1111 Coin: BSC
Number: 1112 Coin: SH
Number: 1113 Coin: EGO
Number: 1114 Coin: ARB
Number: 1115 Coin: URC
Number: 1116 Coin: ORLY
Number: 1117 Coin: STEPS
Number: 1118 Coin: OS76
Number: 1119 Coin: ZNE
Number: 1120 Coin: KNC
Number: 1121 Coin: AGLC
Number: 1122 Coin: CAB
Number: 1123 Coin: IMPS
Number: 1124 Coin: BIOS
Number: 1125 Coin: IBANK
Number: 1126 Coin: XOC
Number: 1127 Coin: JOBS
Number: 1128 Coin: ELS
Number: 1129 Coin: LIR
Number: 1130 Coin: CRT
Number: 1131 Coin: HVCO
Number: 1132 Coin: SDP
Number: 1133 Coin: ALTC
Number: 1134 Coin: P7C
Number: 1135 Coin: XRC
Number: 1136 Coin: NODC
Number: 1137 Coin: SANDG
Number: 1138 Coin: FXE
Number: 

Number: 1446 Coin: ACN
Number: 1447 Coin: OMC
Number: 1448 Coin: MONEY
Number: 1449 Coin: UTA
Number: 1450 Coin: XVE
Number: 1451 Coin: BLAZR
Number: 1452 Coin: DISK
Number: 1453 Coin: UNC
Number: 1454 Coin: ASN
Number: 1455 Coin: SNAKE
Number: 1456 Coin: SHELL
Number: 1457 Coin: LAZ
Number: 1458 Coin: POKE
Number: 1459 Coin: CC
Number: 1460 Coin: DBG
Number: 1461 Coin: TCR
Number: 1462 Coin: RUNNERS
Number: 1463 Coin: XAU
Number: 1464 Coin: LEPEN
Number: 1465 Coin: TERA
Number: 1466 Coin: PRM
Number: 1467 Coin: RUPX
Number: 1468 Coin: MMXVI
Number: 1469 Coin: OP
Number: 1470 Coin: CYC
Number: 1471 Coin: AV
Number: 1472 Coin: OPES
Number: 1473 Coin: EGG
Number: 1474 Coin: LTH
Number: 1475 Coin: CHEAP
Number: 1476 Coin: LKC
Number: 1477 Coin: HYTV
Number: 1478 Coin: FUTC
Number: 1479 Coin: GML
Number: 1480 Coin: EBIT
Number: 1481 Coin: MBL
Number: 1482 Coin: DCRE
Number: 1483 Coin: X2
Number: 1484 Coin: ELTC2
Number: 1485 Coin: AXIOM
Number: 1486 Coin: BAC
Number: 1487 Coin: STEX
Number

NameError: name 'checkannouncement' is not defined

In [10]:
HDLB

NameError: name 'HDLB' is not defined